In [9]:
from gpkit import Model, Variable, units
from gpkit.shortcuts import Var, Model
from cost import DAPCA4Cost
from breguet_range import BreguetRange

class RvsC(Model):
    def setup(self):
        cost = DAPCA4Cost()
        rng = BreguetRange()
        
        C_fly = cost["C_{fly}"]
        R = rng["R"]
        
        R_min = Var("R_{min}", 200, "nautical_miles", "Minimum Range")
        
        objective = C_fly # minimize cost
        constraints = [R >= R_min]
        
        model = Model(objective, constraints)

        for subm in [cost, rng]:
            model = model & subm
            
        return model
    
    def test(self):
        self.solve(verbosity=0)
    
print RvsC().solve(verbosity=0).table(["cost", "freevariables"])
        


The 'setup' method of this model had an error.


KeyError: 'C_{fly}'